# The Right Fit
Overfitting is where a model matches the training data almost perfectly, so when the model is being created, it looks like a near 100% accuracy rate; however, it does poorly in validation and other new data because it is perfectly fitted to the data that it was trained on. Underfitting is the opposite, where a when the model is being built, the accuracies are very high, similar to overfitting, but the model is missing crucial categories that define trends seen in the data and just has a very specific set of comparisons.

Neither are desirable in a model as it greatly reduces the accuracy of the model when it is used on either training data or a larger set of data that was not used for the initial creation of the model. This eliminates the models usefulness as it will make inaccurate predictions about what it is processing because it is missing a broader understanding of trends/patterns in the data.

![](https://raw.githubusercontent.com/UncertainQubit/firstrepo/master/R3ywQsR.png)

An example of overfitting can be seen in the model below. When you test the model purely on the data it was trained with it performs extremely well (extremely close estimates to actual housing costs). However, when tested on separate validation data the model has an extremely large margin of error. Below we will explore ways to help identify and correct overfitting and underfitting in your model.

In [13]:
# Code you have previously used to load data
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


# Path of the file to read
iowa_file_path = 'https://raw.githubusercontent.com/UncertainQubit/firstrepo/master/train.csv'

home_data = pd.read_csv(iowa_file_path)
# Create target object and call it y
y = home_data.SalePrice
# Create X
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Specify Model
iowa_model = DecisionTreeRegressor(max_leaf_nodes = 10000)
# Fit Model
iowa_model.fit(train_X, train_y)

# Make training predictions and calculate mean absolute error
train_predictions = iowa_model.predict(train_X)
train_mae = mean_absolute_error(train_predictions, train_y)
print("Training Mean Average Error (MAE): ${:,.0f}".format(train_mae))

# Make validation predipoctions and calculate mean absolute error
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation Mean Average Error (MAE): ${:,.0f}".format(val_mae))

Training Mean Average Error (MAE): $62
Validation Mean Average Error (MAE): $29,049


# Hyperparameter Analysis
One way to identify overfitting/underfitting is if your model performs great with training data but poorly with new testing data which is why it is important to have both a large training dataset and a large testing dataset. One of the primary ways ML engineers work to address this problem is through something called hyperparameter analysis. Essential this means trying a bunch of different values for various parameters within a model (such as the number of end nodes). This can either be done as a semi-automated process (like we will below) or as a fully automated process 

# Exercise
We have created the function `get_mae` for you which will generate the MAE for different versions of our model. Code for this function can be viewed in the cell below.

In [5]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

## Step 1: Compare Different Tree Sizes
Write a loop that tries the following values for *max_leaf_nodes* from a set of possible values.

Call the *get_mae* function on each value of max_leaf_nodes. Store the output in some way that allows you to select the value of `max_leaf_nodes` that gives the most accurate model on your data.

In [6]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500, 1000, 2000]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
_
scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)
print(best_tree_size)
# Store the best value of max_leaf_nodes (it will be either 5, 25, 50, 100, 250 or 500)
best_tree_size = 100

NameError: name 'get_mae' is not defined

## Step 2: Fit Model Using All Data
You know the best tree size. If you were going to deploy this model in practice, you would make it even more accurate by using all of the data and keeping that tree size.  That is, you don't need to hold out the validation data now that you've made all your modeling decisions.

In [7]:
# Fill in argument to make optimal size and uncomment
final_model = DecisionTreeRegressor(max_leaf_nodes = best_tree_size, random_state = 0)

# fit the final model and uncomment the next two lines
final_model.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=100, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

You've tuned this model and improved your results. But we are still using Decision Tree models, which are not very sophisticated by modern machine learning standards. In the next step you will learn to use Random Forests to improve your models even more.

# Great Job!